# Common Table Expressions (CTE)
Sub Query terpisah yang disimpan sebagai tabel sementara di query dan bisa digunakan berulang kali.
Keyword: WITH nama_tabel AS (query)

In [62]:
import mysql.connector
import pandas as pd

# CREATE CONNECTION
mydb = mysql.connector.connect(
    host='localhost',
    user='root',
    passwd='Password_untuk_SQL',
    database='sakila')

# CREATE ACCES TO DATABASE
mycursor = mydb.cursor()

# QUERY
query='SELECT * FROM film;'

# EXECUTE QUERY
mycursor.execute(query)

# SAVE RESULT
result = mycursor.fetchall()

# CONVERT INTO DATAFRAME
# pd.DataFrame(result, columns=mycursor.column_names)

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"{Deleted Scenes, Behind the Scenes}",2006-02-15 05:03:42
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,None,3,4.99,48,12.99,G,"{Trailers, Deleted Scenes}",2006-02-15 05:03:42
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,None,7,2.99,50,18.99,NC-17,"{Trailers, Deleted Scenes}",2006-02-15 05:03:42
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,None,5,2.99,117,26.99,G,"{Behind the Scenes, Commentaries}",2006-02-15 05:03:42
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,None,6,2.99,130,22.99,G,{Deleted Scenes},2006-02-15 05:03:42
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,YOUNG LANGUAGE,A Unbelieveable Yarn of a Boat And a Database ...,2006,1,None,6,0.99,183,9.99,G,"{Trailers, Behind the Scenes}",2006-02-15 05:03:42
996,997,YOUTH KICK,A Touching Drama of a Teacher And a Cat who mu...,2006,1,None,4,0.99,179,14.99,NC-17,"{Trailers, Behind the Scenes}",2006-02-15 05:03:42
997,998,ZHIVAGO CORE,A Fateful Yarn of a Composer And a Man who mus...,2006,1,None,6,0.99,105,10.99,NC-17,{Deleted Scenes},2006-02-15 05:03:42
998,999,ZOOLANDER FICTION,A Fateful Reflection of a Waitress And a Boat ...,2006,1,None,5,2.99,101,28.99,R,"{Trailers, Deleted Scenes}",2006-02-15 05:03:42


In [3]:
def sql(query):
    mycursor.execute(query)
    result=mycursor.fetchall()
    df=pd.DataFrame(result,columns=mycursor.column_names)
    return df

In [9]:
# Menampilkan film_id dan rental durasi (lama pinjam) dari film film yang panjang tayang (length) nya diatas rata-rata

# CARA SUB QUERY
# sql('''
# SELECT film_id, rental_duration
#     FROM film
#     WHERE length>(SELECT AVG(length) FROM film)
#     ORDER BY rental_duration DESC;
# ''')

# CARA CTE
sql('''
WITH tabel1 AS (SELECT AVG(length) AS rata_lama FROM film)
SELECT film_id, rental_duration
    FROM film
    WHERE length>(SELECT rata_lama from tabel1)
    ORDER BY 2 DESC;
''')

,film_id,rental_duration
0,27,7
1,36,7
2,70,7
3,80,7
4,88,7
...,...,...
484,946,3
485,973,3
486,984,3
487,987,3


In [10]:
# Menampilkan film yang lamanya di antara rata-rata lama film dengan rating 'g' dan 'pg-13'
sql('''
SELECT rating, AVG(length) AS avg_length
    FROM film
    GROUP BY rating
''')

,rating,avg_length
0,PG,112.0052
1,G,111.0506
2,NC-17,113.2286
3,PG-13,120.4439
4,R,118.6615


In [13]:
# CARA CTE
sql('''
WITH tabel1 AS (SELECT rating, AVG(length) AS avg_length
                    FROM film
                    GROUP BY rating)
SELECT title, length
    FROM film
    WHERE length BETWEEN (SELECT avg_length FROM tabel1 WHERE rating='g')
                     AND (SELECT avg_length FROM tabel1 WHERE rating='pg-13')
''')

,title,length
0,AFFAIR PREJUDICE,117
1,ALABAMA DEVIL,114
2,AMADEUS HOLY,113
3,APOCALYPSE FLAMINGOS,119
4,ATTACKS HATE,113
...,...,...
65,TROOPERS METAL,115
66,UNTOUCHABLES SUNRISE,120
67,WAIT CIDER,112
68,WATERSHIP FRONTIER,112


In [15]:
## Kita bisa membuat lebih dari 1 CTE dalam 1 query
# Menampilkan judul fim, durasi film dan lama peminjaman dari film dengan lama peminjaman diatas rata rata dan durasi film dibawah rata rata
sql('''
WITH tabel1 AS (SELECT AVG(rental_duration) AS rata_pinjam FROM film),
     tabel2 AS (SELECT AVG(length) AS rata_lama FROM film)
SELECT title, length, rental_duration
    FROM film
    WHERE rental_duration > (SELECT rata_pinjam FROM tabel1) AND length < (SELECT rata_lama FROM tabel2)
''')

,title,length,rental_duration
0,ACADEMY DINOSAUR,86,6
1,ADAPTATION HOLES,50,7
2,AIRPLANE SIERRA,62,6
3,AIRPORT POLLOCK,54,6
4,ALADDIN CALENDAR,63,6
...,...,...,...
288,WONKA SEA,85,6
289,WYOMING STORM,100,6
290,YENTL IDAHO,86,5
291,ZHIVAGO CORE,105,6


# 2 WINDOW FUNCTION
### 2.1 OVER PARTITION

In [17]:
# Menampilkan rata rata rental duration film di sakila
sql('SELECT AVG(rental_duration) FROM film')

,AVG(rental_duration)
0,4.9850


In [23]:
# OVER()
# Menampilkan judul dilm, rental duration dan rata rata rental duration semua film
sql('''
SELECT title, rental_duration, AVG(rental_duration) OVER() AS rata_durasi FROM film
''')
# dengan over(), rata_durasi muncul sebagai kolom baru

,title,rental_duration,rata_durasi
0,ACADEMY DINOSAUR,6,4.9850
1,ACE GOLDFINGER,3,4.9850
2,ADAPTATION HOLES,7,4.9850
3,AFFAIR PREJUDICE,5,4.9850
4,AFRICAN EGG,6,4.9850
...,...,...,...
995,YOUNG LANGUAGE,6,4.9850
996,YOUTH KICK,4,4.9850
997,ZHIVAGO CORE,6,4.9850
998,ZOOLANDER FICTION,5,4.9850


In [26]:
# Menampilkan judul dilm, rental duration dan rata rata rental duration semua film dan selisih durasi rental film  dengan rata rata

sql('''
SELECT title, rental_duration, AVG(rental_duration) OVER() AS rata_durasi, 
rental_duration-AVG(rental_duration) OVER() AS selisih_durasi
FROM film
''')

,title,rental_duration,rata_durasi,selisih_durasi
0,ACADEMY DINOSAUR,6,4.9850,1.0150
1,ACE GOLDFINGER,3,4.9850,-1.9850
2,ADAPTATION HOLES,7,4.9850,2.0150
3,AFFAIR PREJUDICE,5,4.9850,0.0150
4,AFRICAN EGG,6,4.9850,1.0150
...,...,...,...,...
995,YOUNG LANGUAGE,6,4.9850,1.0150
996,YOUTH KICK,4,4.9850,-0.9850
997,ZHIVAGO CORE,6,4.9850,1.0150
998,ZOOLANDER FICTION,5,4.9850,0.0150


### OVER(PARTITION BY)

In [27]:
# OVER(PARTITION BY)
# rata2 rental rate berdasarkan rating
sql('''
SELECT rating, AVG(rental_rate) FROM film GROUP BY rating
''')

,rating,AVG(rental_rate)
0,PG,3.051856
1,G,2.888876
2,NC-17,2.970952
3,PG-13,3.034843
4,R,2.938718


In [29]:
# menampilkan judul, rating film, harga sewa dan rata2 harga sewa berdasarkan rating

sql('''
SELECT title, rating, rental_rate, AVG(rental_rate) OVER(PARTITION BY rating) AS rata_harga_sewa
FROM film
''')

,title,rating,rental_rate,rata_harga_sewa
0,ACE GOLDFINGER,G,4.99,2.888876
1,AFFAIR PREJUDICE,G,2.99,2.888876
2,AFRICAN EGG,G,2.99,2.888876
3,ALAMO VIDEOTAPE,G,0.99,2.888876
4,AMISTAD MIDSUMMER,G,2.99,2.888876
...,...,...,...,...
995,WONKA SEA,NC-17,2.99,2.970952
996,WRATH MILE,NC-17,0.99,2.970952
997,YOUTH KICK,NC-17,0.99,2.970952
998,ZHIVAGO CORE,NC-17,0.99,2.970952


In [30]:
# menampilkan judul, rating film, harga sewa dan rata2 harga sewa berdasarkan rating dan selisih harga sewa dengan rata-rata harga sewa

sql('''
SELECT title, rating, rental_rate, AVG(rental_rate) OVER(PARTITION BY rating) AS rata_harga_sewa,
rental_rate-AVG(rental_rate) OVER(PARTITION BY RATING) AS selisih
FROM film
''')

,title,rating,rental_rate,rata_harga_sewa,selisih
0,ACE GOLDFINGER,G,4.99,2.888876,2.101124
1,AFFAIR PREJUDICE,G,2.99,2.888876,0.101124
2,AFRICAN EGG,G,2.99,2.888876,0.101124
3,ALAMO VIDEOTAPE,G,0.99,2.888876,-1.898876
4,AMISTAD MIDSUMMER,G,2.99,2.888876,0.101124
...,...,...,...,...,...
995,WONKA SEA,NC-17,2.99,2.970952,0.019048
996,WRATH MILE,NC-17,0.99,2.970952,-1.980952
997,YOUTH KICK,NC-17,0.99,2.970952,-1.980952
998,ZHIVAGO CORE,NC-17,0.99,2.970952,-1.980952


### ROW_NUMBER()
memberi nomor berdasarkan barisnya

In [34]:
sql('''
SELECT title, rating, ROW_NUMBER() OVER(PARTITION BY rating) AS baris_by_rating,
ROW_NUMBER() OVER() AS baris_ke
FROM film
''')

,title,rating,baris_by_rating,baris_ke
0,ACE GOLDFINGER,G,1,1
1,AFFAIR PREJUDICE,G,2,2
2,AFRICAN EGG,G,3,3
3,ALAMO VIDEOTAPE,G,4,4
4,AMISTAD MIDSUMMER,G,5,5
...,...,...,...,...
995,WONKA SEA,NC-17,206,996
996,WRATH MILE,NC-17,207,997
997,YOUTH KICK,NC-17,208,998
998,ZHIVAGO CORE,NC-17,209,999


In [37]:
sql('''
WITH tabel1 AS (
SELECT title, rating, 
ROW_NUMBER() OVER(PARTITION BY rating) AS baris_by_rating,
ROW_NUMBER() OVER() AS baris_ke
FROM film
)
SELECT * FROM tabel1
WHERE baris_by_rating<=3

''')

,title,rating,baris_by_rating,baris_ke
0,ACE GOLDFINGER,G,1,1
1,AFFAIR PREJUDICE,G,2,2
2,AFRICAN EGG,G,3,3
3,ACADEMY DINOSAUR,PG,1,179
4,AGENT TRUMAN,PG,2,180
5,ALASKA PHANTOM,PG,3,181
6,AIRPLANE SIERRA,PG-13,1,373
7,ALABAMA DEVIL,PG-13,2,374
8,ALTER VICTORY,PG-13,3,375
9,AIRPORT POLLOCK,R,1,596


### RANK() & DENSE_RANK()
Rank -> membuat ranking berdasarkan nilai tertentu

In [41]:
sql('''
SELECT title, rating, rental_rate, replacement_cost,
DENSE_RANK() OVER(ORDER BY replacement_cost) AS rank_cost
FROM film
''')

,title,rating,rental_rate,replacement_cost,rank_cost
0,ANACONDA CONFESSIONS,R,0.99,9.99,1
1,CIDER DESIRE,PG,2.99,9.99,1
2,CONTROL ANTHEM,G,4.99,9.99,1
3,DAISY MENAGERIE,G,4.99,9.99,1
4,DELIVERANCE MULHOLLAND,R,0.99,9.99,1
...,...,...,...,...,...
995,TRACY CIDER,G,0.99,29.99,21
996,UNCUT SUICIDES,PG-13,2.99,29.99,21
997,VIRGIN DAISY,PG-13,4.99,29.99,21
998,WEST LION,G,4.99,29.99,21


In [54]:
pd.set_option("display.max_rows", 10)
sql('''
SELECT title, rating, rental_rate, replacement_cost,
DENSE_RANK() OVER(PARTITION BY rating ORDER BY replacement_cost) AS rank_cost
FROM film
''')

,title,rating,rental_rate,replacement_cost,rank_cost
0,CONTROL ANTHEM,G,4.99,9.99,1
1,DAISY MENAGERIE,G,4.99,9.99,1
2,DUDE BLINDNESS,G,4.99,9.99,1
3,ENCINO ELF,G,0.99,9.99,1
4,HEARTBREAKERS BRIGHT,G,4.99,9.99,1
...,...,...,...,...,...
995,IDOLS SNATCHERS,NC-17,2.99,29.99,21
996,JERICHO MULAN,NC-17,2.99,29.99,21
997,PATIENT SISTER,NC-17,0.99,29.99,21
998,PRINCESS GIANT,NC-17,2.99,29.99,21


In [52]:
# Menampilkan ranking rata-rata replacement cost berdasarkan ratingnya
# sql('''
# SELECT rating, AVG(replacement_cost) AS rata_cost,
# RANK() OVER(ORDER BY AVG(replacement_cost) DESC) AS ranking
# FROM film
# GROUP BY rating
# ''')

sql('''
SELECT rating, AVG(replacement_cost) AS rata_cost,
RANK() OVER(ORDER BY AVG(replacement_cost) DESC) AS ranking
FROM film
GROUP BY rating
''')

# menampilkan raing, rata2 cost berdasarkan rating dan rankingnya

,rating,rata_cost,ranking
0,PG-13,20.402556,1
1,R,20.231026,2
2,NC-17,20.137619,3
3,G,20.124831,4
4,PG,18.959072,5


### NTILE()
membuat kolom baru berisikan sejumlah nomor sesuai yang kita tentukan. Pembagian nomor berdasarkan urutan data.

Misal ada 20 data kita mau membuat 4 ntile maka
5 data pertama dengan nilai terkecil (default) akan diberi angka 1
5 data berikutnya 2 dan nilai data dengan nilai terbesar 4

In [55]:
sql('''
SELECT title, replacement_cost,
NTILE(4) OVER(ORDER BY replacement_cost DESC) AS quatile_cost
FROM film
''')

,title,replacement_cost,quatile_cost
0,ARABIA DOGMA,29.99,1
1,BALLROOM MOCKINGBIRD,29.99,1
2,BLINDNESS GUN,29.99,1
3,BONNIE HOLOCAUST,29.99,1
4,CHARIOTS CONSPIRACY,29.99,1
...,...,...,...
995,TROJAN TOMORROW,9.99,4
996,TRUMAN CRAZY,9.99,4
997,VALENTINE VANISHING,9.99,4
998,WAIT CIDER,9.99,4


### SLIDING WINDOW
untuk mencari nilai aggregasi dari beberapa data berurutan, biasanya dipakai untuk mencari moving average atau cumulative sum

In [60]:
sql('''
WITH tabel1 AS (SELECT rating, count(*) AS jumlah_film FROM film GROUP BY rating)
SELECT rating, jumlah_film, SUM(jumlah_film) OVER(ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS total_cumulative,
AVG(jumlah_film) OVER(ROWS BETWEEN CURRENT ROW AND 1 FOLLOWING) AS rata_bergerak
FROM tabel1
''')

# Penggunaan following dan preciding didahului dengan angka/berapa baris

,rating,jumlah_film,total_cumulative,rata_bergerak
0,PG,194,194,186.0000
1,G,178,372,194.0000
2,NC-17,210,582,216.5000
3,PG-13,223,805,209.0000
4,R,195,1000,195.0000
